# MCP Multi-Server Agent with Claude Sonnet 4

A tool-calling agent using Databricks' Claude Sonnet 4 endpoint with multiple MCP server capabilities.

## What This Agent Does

🔍 **Query Databricks System Tables** - Ask natural language questions about your Databricks usage  
🔧 **Execute Unity Catalog Functions** - Call your custom functions seamlessly through natural language  
🤖 **Claude Sonnet 4 Powered** - Uses Databricks' most advanced LLM endpoint  
🎯 **Multi-MCP Server Support** - Extensible architecture for multiple MCP servers  
🚀 **Production Ready** - Deploy to Databricks Playground via MLflow

## Setup

In [ ]:
# Install dependencies
%pip install -r requirements.txt
print("✅ Dependencies installed!")

In [ ]:
import asyncio
import os
import sys
sys.path.insert(0, '.')

# Remove conflicting tokens
if "DATABRICKS_TOKEN" in os.environ:
    del os.environ["DATABRICKS_TOKEN"]

from src.agent import SingleTurnMCPAgent
from config import config, validate_oauth_setup

print("✅ Imports complete!")
print(f"🔗 LLM Endpoint: {config.llm_endpoint_name}")
print(f"🌐 Workspace: {config.databricks_host}")
print(f"🗂️ Genie Space ID: {config.genie_space_id}")
print(f"📚 Unity Catalog: {config.unity_catalog_catalog}.{config.unity_catalog_schema}")

## Configuration Test

Verify OAuth authentication is working:

In [ ]:
# Test OAuth configuration
print("🧪 Testing OAuth Configuration...")
is_valid = validate_oauth_setup()

if is_valid:
    print("✅ OAuth configuration is ready!")
    print("🚀 You can now use the MCP Genie agent")
else:
    print("⚠️ Please configure OAuth in .env file before proceeding")
    print("Required variables: DATABRICKS_HOST, DATABRICKS_CLIENT_ID, DATABRICKS_CLIENT_SECRET, GENIE_SPACE_ID")

## Create Multi-MCP Agent

The agent automatically connects to both Genie (system tables) and Unity Catalog (your functions):

In [ ]:
# Configure both MCP servers
server_configs = [
    {
        'type': 'genie',
        'url': config.genie_server_url,
        'name': 'genie'
    },
    {
        'type': 'unity_catalog',
        'catalog': config.unity_catalog_catalog,
        'schema': config.unity_catalog_schema,
        'name': 'unity_catalog'
    }
]

agent = SingleTurnMCPAgent(
    llm_endpoint=config.llm_endpoint_name,
    system_prompt=config.system_prompt,
    server_configs=server_configs
)

print("✅ Multi-MCP Agent created successfully!")
print(f"📊 Genie Space: {config.genie_space_id}")
print(f"📚 Unity Catalog: {config.unity_catalog_catalog}.{config.unity_catalog_schema}")
print("💡 To use a different Unity Catalog schema, set UNITY_CATALOG_CATALOG and UNITY_CATALOG_SCHEMA environment variables")

## Test Queries

Ask questions about your Databricks usage or call Unity Catalog functions:

In [ ]:
def query_agent(question):
    """Helper function to query the agent."""
    class MockRequest:
        def __init__(self, message):
            self.messages = [{'content': message}]
    
    request = MockRequest(question)
    response = agent.invoke(request)
    return response.output[0].content[0]['text']

# Test query
question = "How many queries were executed in the past 7 days?"
print(f"🔍 Question: {question}")
answer = query_agent(question)
print(f"📝 Answer: {answer}")

In [ ]:
# Try more queries - mix of Genie (system tables) and Unity Catalog functions
test_questions = [
    "What are the most expensive clusters by compute cost?",
    "Can you add the numbers 15.5 and 24.7 using the add_numbers function?", 
    "Show me the top 5 users by activity",
    "What Unity Catalog functions are available to me?"
]

for question in test_questions:
    print(f"\n🔍 Question: {question}")
    try:
        answer = query_agent(question)
        # Show first 200 characters
        display_answer = answer[:200] + "..." if len(answer) > 200 else answer
        print(f"📝 Answer: {display_answer}")
    except Exception as e:
        print(f"❌ Error: {e}")

## Interactive Mode

Ask your own questions:

In [ ]:
# Interactive query - modify this cell to ask your own questions
my_question = "What functions can I call in Unity Catalog and how many queries were executed this week?"

print(f"🔍 Your question: {my_question}")
answer = query_agent(my_question)
print(f"📝 Answer: {answer}")

## Deploy to Databricks Playground

To deploy this agent for organization-wide use:

In [ ]:
# Deploy the agent (uncomment to run)
print("🚀 To deploy this agent to Databricks Playground:")
print("")
print("Option 1: Run deployment script")
print("  python deploy_final.py")
print("")
print("Option 2: Deploy from notebook")
print("  # Uncomment the lines below:")
print("  # from deploy_final import deploy_final_agent")
print("  # deploy_final_agent()")
print("")
print("After deployment, find your agent in Databricks Playground!")

## 🎉 Ready!

Your Multi-MCP Agent is working and ready to analyze Databricks data and execute your functions!

### Example Questions to Try:

#### **Genie (System Tables)**
- "How many queries were executed over the past 7 days?"
- "What are the most expensive clusters by compute cost?"
- "Show me the top SQL queries by execution time"
- "Which users are most active in this workspace?"

#### **Unity Catalog Functions** 
- "Add 25.5 and 17.3 using the add_numbers function"
- "Calculate compound interest for $1000 at 5% for 10 years"
- "What sentiment is the text 'I love this product'?"
- "What Unity Catalog functions are available?"

#### **Combined Queries**
- "Show me system usage data and then calculate a percentage"
- "Get cluster costs and format the top result as currency"

The agent intelligently routes queries to the appropriate MCP server! 🚀

### **Configuration**
- **Unity Catalog Schema**: `{config.unity_catalog_catalog}.{config.unity_catalog_schema}`
- **Change schema**: Set `UNITY_CATALOG_CATALOG` and `UNITY_CATALOG_SCHEMA` environment variables